Importing the all necessary libraries

In [ ]:
import os 
import gc
import cv2
import math 
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import pandas as pd
 
import tensorflow as tf 
# from keras.utils import plot_model 
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import *
from sklearn.model_selection import train_test_split
from sklearn.metrics import make_scorer, accuracy_score, roc_auc_score, roc_curve 

Read images and perform some augmentation on image to increase the dataset

In [ ]:
def read_image(filepath): 
    return cv2.imread(os.path.join(train_dir, filepath)) 

def resize_image(image, image_size): 
    return cv2.resize(image.copy(), image_size, interpolation = cv2.INTER_AREA)  

def get_mat(rotation, shear, height_zoom, width_zoom, height_shift, width_shift): 
    # returns 3x3 transformmatrix which transforms indicies 
        
    # CONVERT DEGREES TO RADIANS 
    rotation = math.pi * rotation / 180.
    shear = math.pi * shear / 180.
    
    # ROTATION MATRIX
    c1 = tf.math.cos(rotation)
    s1 = tf.math.sin(rotation)
    one = tf.constant([1],dtype='float32')
    zero = tf.constant([0],dtype='float32')
    rotation_matrix = tf.reshape( tf.concat([c1,s1,zero, -s1,c1,zero, zero,zero,one],axis=0),[3,3] )
        
    # SHEAR MATRIX
    c2 = tf.math.cos(shear)
    s2 = tf.math.sin(shear)
    shear_matrix = tf.reshape( tf.concat([one,s2,zero, zero,c2,zero, zero,zero,one],axis=0),[3,3] )    
    
    # ZOOM MATRIX
    zoom_matrix = tf.reshape( tf.concat([one/height_zoom,zero,zero, zero,one/width_zoom,zero, zero,zero,one],axis=0),[3,3] )
    
    # SHIFT MATRIX
    shift_matrix = tf.reshape( tf.concat([one,zero,height_shift, zero,one,width_shift, zero,zero,one],axis=0),[3,3] )
    
    return K.dot(K.dot(rotation_matrix, shear_matrix), K.dot(zoom_matrix, shift_matrix))

def transform(image,label):
    # input image - is one image of size [dim,dim,3] not a batch of [b,dim,dim,3]
    # output - image randomly rotated, sheared, zoomed, and shifted
    DIM = IMAGE_SIZE
    XDIM = DIM%2 #fix for size 331
    
    rot = 15. * tf.random.normal([1],dtype='float32')
    shr = 5. * tf.random.normal([1],dtype='float32') 
    h_zoom = 1.0 + tf.random.normal([1],dtype='float32')/10.
    w_zoom = 1.0 + tf.random.normal([1],dtype='float32')/10.
    h_shift = 16. * tf.random.normal([1],dtype='float32') 
    w_shift = 16. * tf.random.normal([1],dtype='float32') 
  
    # GET TRANSFORMATION MATRIX
    m = get_mat(rot,shr,h_zoom,w_zoom,h_shift,w_shift) 

    # LIST DESTINATION PIXEL INDICES
    x = tf.repeat( tf.range(DIM//2,-DIM//2,-1), DIM )
    y = tf.tile( tf.range(-DIM//2,DIM//2),[DIM] )
    z = tf.ones([DIM*DIM],dtype='int32')
    idx = tf.stack( [x,y,z] )
    
    # ROTATE DESTINATION PIXELS ONTO ORIGIN PIXELS
    idx2 = K.dot(m,tf.cast(idx,dtype='float32'))
    idx2 = K.cast(idx2,dtype='int32')
    idx2 = K.clip(idx2,-DIM//2+XDIM+1,DIM//2)
    
    # FIND ORIGIN PIXEL VALUES           
    idx3 = tf.stack( [DIM//2-idx2[0,], DIM//2-1+idx2[1,]] )
    d = tf.gather_nd(image,tf.transpose(idx3))
        
    return tf.reshape(d,[DIM,DIM,3]),label

def get_training_dataset(dataset, do_aug = True):
    if do_aug: 
        dataset = dataset.map(transform, num_parallel_calls = AUTO)
    dataset = dataset.repeat() # the training dataset must repeat for several epochs
    dataset = dataset.shuffle(2048)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.prefetch(AUTO) # prefetch next batch while training (autotune prefetch buffer size)
    return dataset

def get_validation_dataset(dataset):
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.cache()
    dataset = dataset.prefetch(AUTO) # prefetch next batch while training (autotune prefetch buffer size)
    return dataset

Mount the drive to access the data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')  

Mounted at /content/drive


Unzip the file 

In [ ]:
!unzip -uq '/content/drive/MyDrive/ml and ai/mjorpro.zip' 

give the path for the images folder

In [ ]:
train_dir = os.path.join('/content/mjorpro/') 

Binary classes(cracked and uncracked)

In [ ]:
disease_types=['manatees', 'whales'] 

In [ ]:
train_data = []
for defects_id, sp in enumerate(disease_types):
    for file in os.listdir(os.path.join(train_dir, sp)):
        train_data.append(['{}/{}'.format(sp, file), defects_id, sp])      
train = pd.DataFrame(train_data, columns=['File', 'DiseaseID','Disease Type']) 

Resize the image

In [ ]:
IMAGE_SIZE = 64
X = np.zeros((train.shape[0], IMAGE_SIZE, IMAGE_SIZE, 3))
for i, file in tqdm(enumerate(train['File'].values), total = len(train)):
    image = read_image(file)
    if image is not None:
        X[i] = resize_image(image, (IMAGE_SIZE, IMAGE_SIZE))
X /= 255.
y = train['DiseaseID'].values
print(X.shape) 
print(y.shape)

  0%|          | 0/138 [00:00<?, ?it/s]

(138, 64, 64, 3)
(138,)


splitting the dataset (80 percentage train aand 20 percentage test)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=np.random.randint(1,1000, 1)[0], stratify=y) 

In [ ]:
x_train.shape

(110, 64, 64, 3)

In [ ]:
y_train.shape

(110,)

In [ ]:
x_test.shape

(28, 64, 64, 3)

In [ ]:
y_test.shape

(28,)

Convolutional Neural Network

In [ ]:
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)))
model.add(MaxPool2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu')) 
model.add(MaxPool2D((2, 2))) 
model.add(Flatten()) 
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='sigmoid')) 

here we take two classes so called binary classification, hence we used sigmoid activation function

In [ ]:
model.summary() 

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 62, 62, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 31, 31, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 29, 29, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 14, 14, 64)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 12544)             0         
                                                                 
 dense (Dense)               (None, 128)               1

we trained the model forr 40 epochs

In [ ]:
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy']) 
history = model.fit(x_train, y_train, batch_size=64, epochs=50) 

Epoch 1/50
2/2 [==============================] - 2s 366ms/step - loss: 1.8547 - accuracy: 0.6000
Epoch 2/50
2/2 [==============================] - 1s 352ms/step - loss: 0.7339 - accuracy: 0.6818
Epoch 3/50
2/2 [==============================] - 1s 332ms/step - loss: 1.2398 - accuracy: 0.4818
Epoch 4/50
2/2 [==============================] - 1s 357ms/step - loss: 0.5906 - accuracy: 0.7000
Epoch 5/50
2/2 [==============================] - 1s 355ms/step - loss: 0.5433 - accuracy: 0.6455
Epoch 6/50
2/2 [==============================] - 1s 376ms/step - loss: 0.6316 - accuracy: 0.6273
Epoch 7/50
2/2 [==============================] - 1s 357ms/step - loss: 0.8024 - accuracy: 0.6273
Epoch 8/50
2/2 [==============================] - 1s 337ms/step - loss: 0.4464 - accuracy: 0.9091
Epoch 9/50
2/2 [==============================] - 1s 340ms/step - loss: 0.3017 - accuracy: 0.9091
Epoch 10/50
2/2 [==============================] - 1s 313ms/step - loss: 0.6698 - accuracy: 0.6636
Epoch 11/50
2/2 [==

Testing phase

In [ ]:
model.evaluate(x_test, y_test)  

1/1 [==============================] - 0s 411ms/step - loss: 2.1072 - accuracy: 0.5714


[2.107194185256958, 0.5714285969734192]

In [ ]:
# predict probabilities for test set
yhat_probs = model.predict(x_test, verbose=1)
# predict crisp classes for test set

yhat_classes = (model.predict(x_test) > 0.5).astype("int32") 

1/1 [==============================] - 0s 152ms/step


In [ ]:
yhat_probs = yhat_probs[:, 0]
yhat_classes = yhat_classes[:, 0]  

In [ ]:
yhat_probs

array([6.5046549e-04, 2.0723656e-01, 1.0297596e-03, 2.8522053e-01,
       1.1844664e-04, 8.0689460e-02, 8.7557137e-03, 3.4812023e-05,
       1.9182026e-02, 1.4624259e-01, 7.7862561e-01, 9.3512034e-01,
       1.5833300e-05, 2.5691280e-01, 3.1175534e-06, 7.5003460e-05,
       4.4887485e-05, 6.8261120e-06, 5.8042646e-05, 3.9681196e-03,
       1.2169539e-04, 6.3425303e-04, 7.7840656e-01, 1.0656553e-04,
       2.7004391e-02, 6.3761172e-07, 3.5368495e-05, 8.1212471e-05],
      dtype=float32)

In [ ]:
yhat_classes 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0], dtype=int32)

Perforrmance Metrices

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
# accuracy: (tp + tn) / (p + n)
accuracy = accuracy_score(y_test, yhat_classes)
print('Accuracy: %f' % accuracy)
# precision tp / (tp + fp)
precision = precision_score(y_test, yhat_classes)
print('Precision: %f' % precision) 
# recall: tp / (tp + fn)
recall = recall_score(y_test, yhat_classes) 
print('Recall: %f' % recall)
# f1: 2 tp / (2 tp + fp + fn)
f1 = f1_score(y_test, yhat_classes)
print('F1 score: %f' % f1)

Accuracy: 0.571429
Precision: 1.000000
Recall: 0.200000
F1 score: 0.333333


Confusion maatrix

In [ ]:
from sklearn.metrics import confusion_matrix 
matrix = confusion_matrix(y_test, yhat_classes) 
print(matrix)  

[[13  0]
 [12  3]]
